In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
pubg_train = pd.read_csv('/Users/lumi/Documents/UCLA/Grad/18 FALL/273/Pubg/pudg/all/train_group.csv')

In [ ]:
pubg_train.info()
pubg_train.shape

In [ ]:
ytr = pubg_train['winPlacePerc'].as_matrix()

In [ ]:
dfX = pubg_train.drop(['winPlacePerc', 'winPoints', 'killPoints',
                       'kills', 'assists', 'boosts', 'heals','weaponsAcquired'], axis=1)

Xtrain = dfX.as_matrix()
features = dfX.drop(['groupId', 'matchId'], axis = 1).columns.tolist()

In [ ]:
#regularization with CV
from sklearn import linear_model
from sklearn import model_selection

In [ ]:
from sklearn.model_selection import train_test_split

X_tr0, X_ts0, y_tr, y_ts = train_test_split(Xtrain, ytr, test_size = 0.2, train_size = 0.8)
#X_tr0 and X_ts0 will keep the match id and group id here

In [ ]:
#remove matchid and group id from train and test data
X_tr = np.delete(X_tr0, [0, 1], 1)
X_ts = np.delete(X_ts0, [0, 1], 1)

In [ ]:
#get match id and group id
id_tr = X_tr0[:,[0,1]]
id_ts = X_ts0[:, [0,1]]

In [ ]:
nsamp = Xtrain.shape[0]
nf = len(features)
print('number of sample', nsamp)
print('number of feature', nf)

In [ ]:
from sklearn import preprocessing

X_tr = X_tr.astype(float)
X_tr = preprocessing.scale(X_tr)
X_ts = X_ts.astype(float)
X_ts = preprocessing.scale(X_ts)
#ys = preprocessing.scale(ytr)


In [ ]:
#limit target variable in 0 and 1
def confine_predictions(x):
    for i in range(len(x)):
        if x[i] > 1:
            x[i] = 1
        elif x[i] < 0:
            x[i] = 0

OLS Regression

In [41]:
# Fit linear model 
reg = linear_model.LinearRegression()
reg.fit(X_tr, y_tr)

# Measure MAE for the training set and print
yhat = reg.predict(X_tr)
mae_tr = np.mean(abs(y_tr - yhat))
print('MAE on the training data =', mae_tr)



MAE on the training data = 0.07868786849898912


In [42]:
yhat = reg.predict(X_ts)
confine_predictions(yhat)
yhat_OLS = yhat
mae_OLS = np.mean(abs(y_ts-yhat_OLS))
print('MAE on the test data =', mae_OLS)

MAE on the test data = 0.07630574522086866


Lasso Regression

In [28]:
#Fit Lasso
lassomodel = linear_model.Lasso(alpha=1e-3)
nalpha = 100
alpha_test = np.logspace(-4, 3, nalpha)
mae_ts_lasso = np.zeros(nalpha)

In [31]:
#find optimal alpha
for it, a in enumerate(alpha_test):
        lassomodel.alpha = a
        lassomodel.fit(X_tr, y_tr)
        yhat = lassomodel.predict(X_ts)
        # MAE
        mae_ts_lasso[it] = np.mean(abs(y_ts - yhat))

In [33]:
minmae = min(mae_ts_lasso)
index = np.where(mae_ts_lasso == minmae)
print('the minimum MAE =', minmae)
print('the alpha that minimizes the MAE is', alpha_test[index])

the minimum MAE = 0.07876953007485529
the alpha that minimizes the MAE is [0.0001]


In [34]:
a = alpha_test[index]
lassomodel = linear_model.Lasso(alpha = a)
lassomodel.fit(X_tr, y_tr)
yhat = lassomodel.predict(X_ts)

In [35]:
confine_predictions(yhat)
yhat_lasso = yhat

In [36]:
mae_lasso = np.mean(abs(y_ts - yhat_lasso))
print(mae_lasso)

0.07641082798531426


In [ ]:
coef = lassomodel.coef_
rowname = list(features)
table = pd.DataFrame(coef, index = rowname)
print(table)

In [ ]:
alphas1, coeffs, _ = linear_model.lasso_path(X_tr, y_tr, method='lasso', alpha = alpha_test)
coeffs.shape #predictor * alpha
#match an alpha with a row of coef


for coef_l in coeffs:
    plt.plot(alphas1, coef_l)
    
plt.xlabel('alpha')
plt.ylabel('coef')
plt.legend(rowname, loc='upper right')

In [ ]:
nonzero = (coeffs != 0).astype(int)
nnz = np.sum(nonzero, axis=1)

nonzerotable = pd.DataFrame(nnz, index= rowname)
nonzerotable.columns = ['nnz']
nonzerotable.sort_values(by=['nnz'], ascending=False)

Ridge Regression

In [37]:
ridgemodel = linear_model.Ridge(alpha=1e-3)
nalpha = 100
alpha_test = np.logspace(-4, 3, nalpha)
mae_ts_ridge = np.zeros(nalpha)

In [ ]:
#find optimal alpha
for it, a in enumerate(alpha_test):
        ridgemodel.alpha = a
        ridgemodel.fit(X_tr, y_tr)
        yhat = ridgemodel.predict(X_ts)
        # MAE
        mae_ts_ridge[it] = np.mean(abs(y_ts - yhat))

In [ ]:
minmae = min(mae_ts_ridge)
index = np.where(mae_ts_ridge == minmae)
print('the minimum MAE =', minmae)
print('the alpha that minimizes the MAE is', alpha_test[index])

In [38]:
a = alpha_test[index]
ridgemodel = linear_model.Ridge(alpha = a)
ridgemodel.fit(X_tr, y_tr)
yhat = ridgemodel.predict(X_ts)

In [39]:
confine_predictions(yhat)
yhat_ridge = yhat

In [40]:
mae_ridge = np.mean(abs(y_ts - yhat_ridge))
print(mae_ridge)

0.076305745225682


In [ ]:
coef = ridgemodel.coef_
table = pd.DataFrame(coef, index = rowname)
print(table)

In [31]:
nonzero = (coeffs != 0).astype(int)
nnz = np.sum(nonzero, axis=1)

nonzerotable = pd.DataFrame(nnz, index= rowname)
nonzerotable.columns = ['nnz']
nonzerotable.sort_values(by=['nnz'], ascending=False)

,nnz
walkDistance,99
killPlace,99
itemDuration,91
numGroups,71
rideDistance,63
matchDuration,63
killStreaks,62
revives,46
KADuration,44
headshotKills,40


Random Forest

In [13]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100)

In [14]:
rf.fit(X_tr, y_tr)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [16]:
yhat = rf.predict(X_ts)
confine_predictions(yhat)
yhat_rf = yhat
mae_rf = np.mean(abs(y_ts - yhat_rf))
print('MAE:', mae_rf)

MAE: 0.052245441285688636


Boosting

In [21]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [22]:
boosting = XGBClassifier(n_estimators = 1)

In [ ]:
boosting.fit(X_tr, y_tr)

In [ ]:
yhat = boosting.predict(X_ts)
confine_predictions(yhat)
yhat_boosting = yhat
mae_boosting = np.mean(abs(y_ts - yhat_boosting))
print('MAE:', mae_boosting)

In [17]:
#output test train file
X_tr_output = pd.DataFrame(X_tr)
X_ts_output = pd.DataFrame(X_ts)
y_tr_output = pd.DataFrame(y_tr)
y_ts_output = pd.DataFrame(y_ts)


In [18]:
pd.DataFrame.to_csv(X_tr_output, '/Users/lumi/Documents/UCLA/Grad/18 FALL/273/Pubg/pudg/all/X_tr.csv')

In [19]:
pd.DataFrame.to_csv(X_ts_output, '/Users/lumi/Documents/UCLA/Grad/18 FALL/273/Pubg/pudg/all/X_ts.csv')

In [20]:
pd.DataFrame.to_csv(y_tr_output, '/Users/lumi/Documents/UCLA/Grad/18 FALL/273/Pubg/pudg/all/y_tr.csv')

In [21]:
pd.DataFrame.to_csv(y_ts_output, '/Users/lumi/Documents/UCLA/Grad/18 FALL/273/Pubg/pudg/all/y_ts.csv')

Get tables for all the predictions

In [89]:
matchId = id_ts[:,0].reshape(405349,1)
groupId = id_ts[:,1].reshape(405349,1)
y_ts1 = y_ts.reshape(405349,1)
yhat_OLS1 = yhat_OLS.reshape(405349,1)
yhat_lasso1 = yhat_lasso.reshape(405349,1)
yhat_ridge1 = yhat_ridge.reshape(405349,1)
yhat_rf1 = yhat_rf.reshape(405349,1)

In [92]:
prediction = np.concatenate((matchId, groupId, y_ts1, yhat_OLS1, yhat_lasso1, yhat_ridge1, yhat_rf1), axis = 1)

In [103]:
prediction1 = pd.DataFrame(prediction, columns = ["matchId", "groupId", "y_ts", "OLS", "lasso", "ridge", "rf"])

In [105]:
pd.DataFrame.to_csv(prediction1, '/Users/lumi/Documents/UCLA/Grad/18 FALL/273/Pubg/pudg/all/yhat.csv')